*Copyright 2021 Abhishek Dutta.*

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### COVID-19 waves: $SI^2Q^2R^2VD$ identification over variants

In [ ]:
import pandas as pd
import numpy as np
from scipy import integrate as igt
from scipy import optimize as opt
import plotly.graph_objects as go
import plotly.express as px
#import plotly.io as pio
import datetime as dt

Now, load the data from each .csv file, for USA.

In [ ]:
country = 'US'
# Infections
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
cols = df.columns[4:]
confirmed = df.loc[df['Country/Region']==country, cols].values.flatten()
# Vaccinated
df = pd.read_csv('https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv')
vaccinated = df.loc[df['Country_Region']==country, df.columns[4]].values.flatten()
# Recovered
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
recovered = df.loc[df['Country/Region']==country, cols].values.flatten()
# Deceased
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
deceased = df.loc[df['Country/Region']==country, cols].values.flatten()

Make the length adjustments!

In [ ]:
infected_dly = np.diff(confirmed) # Daily infections
deceased_dly = np.diff(deceased) # Daily deceased
dates = cols.values # Jan 22, 2020 onwards
x = [dt.datetime.strptime(d,'%m/%d/%y').date() for d in dates]
lin = confirmed.shape[0]; nvc = len(vaccinated)
lrv = 326 # US recovery data exists till Dec 13
lvc = lin - nvc # US partially vaccinated from Dec 12
lvf = lvc + 25 # US fully vaccinated from Jan 14
vaccinated = np.pad(vaccinated, (lvc, 0), 'constant')

### Pandemic model with 2 variants
\begin{align}
    &\dot S=-(1-\alpha_T)\frac{\beta_t S(I_1+I_2\beta_v)}{N}-\alpha_T S\\
    &\dot I_1=(1-\alpha_T)\frac{\beta_t SI_1}{N}-(\tau+\rho)I_1\\
    &\dot I_2=(1-\alpha_T)\frac{\beta_t (S+R_1)I_2\beta_v}{N}-(\tau+\rho)I_2\\
    &\dot Q_1=\tau I_1-(\rho+\delta) Q_1\\
    &\dot Q_2=\tau I_2-(\rho+\delta) Q_2\\
    &\dot R_1=\rho(I_1+Q_1)-\alpha_T R_1-(1-\alpha_T)\frac{\beta_t R_1 I_2\beta_v}{N}\\
    &\dot R_2=\rho(I_2+Q_2)-\alpha_T R_2\\
    &\dot V=\alpha_T (S+R_1+R_2)\\
    &\dot D=\delta (Q_1+Q_2)
\end{align}
Step function $\alpha_T,\beta_v\in R_0^+: \alpha_T,\beta_v=0\;\forall t<T,v$.

Sigmoid function for $\beta_t\in R_0^+$ :
$((\underline\beta+\bar\beta)+(\bar\beta-\underline\beta)*tanh((t-t_0)/w))/2$ limits b-a, inflection $t_0$, slope w.

In [ ]:
def siqrvd(t, y, alpha, beta, bt0, tau, rho, delta, btv, ldv):
    S, I1, I2, Q1, Q2, R1, R2, V, D, tI, rQ = y
    alpha_t = alpha * np.heaviside(t-lvf,0)
    beta_v = btv * np.heaviside(t-ldv,0)
    if t < lock: beta_t = bt0 # till lockdown begins
    else: beta_t = ((beta+bt0)+(bt0-beta)*np.tanh((t-300)/100))/2
    if t > ctrl:
      #alpha_t = max(1-(rho+tau)*N/(beta_t*S),1-(rho+tau)*N/(beta_t*(S+R1)*beta_v),0)
      alpha_t = amax
    dSdt = -(1-alpha_t)*beta_t*S*(I1+I2*beta_v)/N - alpha_t*S
    dI1dt = (1-alpha_t)*beta_t*S*I1/N - (tau+rho)*I1
    dI2dt = (1-alpha_t)*beta_t*(S+R1)*I2*beta_v/N - (tau+rho)*I2
    dQ1dt = tau*I1 - (rho+delta)*Q1
    dQ2dt = tau*I2 - (rho+delta)*Q2
    dR1dt = rho*(I1+Q1) - alpha_t*R1 - (1-alpha_t)*beta_t*R1*I2*beta_v/N
    dR2dt = rho*(I2+Q2) - alpha_t*R2
    dVdt = alpha_t*(S+R1+R2)
    dDdt = delta*(Q1+Q2)
    tauI = tau*(I1+I2)
    rhoQ = rho*(Q1+Q2)
    return dSdt, dI1dt, dI2dt, dQ1dt, dQ2dt, dR1dt, dR2dt, dVdt, dDdt, tauI, rhoQ

### Fit model to data
We will use the historical data to find the optimal parameters ($\alpha,\beta,\beta_0,\tau,\rho,\delta,I_0$) of the SIQVRD model.

In [ ]:
# Optimization cost: cumulative error
cm = max(confirmed); vm = max(vaccinated); rm = max(recovered); dm = max(deceased) #normalize
def cost(vec):
    alpha,beta,bt0,tau,rho,delta,btv,ldv,i0 = vec; y0 = [N-i0-q0-v0-r0-d0,i0,1,q0,0,r0,0,v0,d0,n0,g0]
    sol = igt.solve_ivp(siqrvd, [0,t_f], y0, method='Radau', args=(alpha,beta,bt0,tau,rho,delta,btv,ldv), t_eval=t_eval)
    error = np.sum((confirmed-sol.y[9])**2)/(lin*cm**2) + np.sum((vaccinated[lvf:]-sol.y[7,lvf:])**2)/(nvc*vm**2)
    + np.sum((recovered[:lrv]-sol.y[10,:lrv])**2)/(lrv*rm**2) + np.sum((deceased-sol.y[8])**2)/(lin*dm**2)
    return error

In [ ]:
N = 331e6; t_f = lin; t_eval = np.arange(0,t_f,1); ctrl = 1e4
n0 = confirmed[0]; v0 = vaccinated[0]; r0 = recovered[0]; d0 = deceased[0]
q0 = n0-r0-d0; g0 = r0; lock = 60 # ldv = 350 delta variant
init = [0.005,0.001,0.25,0.001,0.01,0.0002,1,375,10]
bnd = ((0,0.01),(0,0.1),(0.1,0.6),(0,0.1),(0,0.2),(0,0.2),(1,1.5),(300,400),(0,10))

# 1. Local optimization - unconstrained
ret = opt.minimize(fun=cost, x0=init, method='Nelder-Mead')
# 2. Global optimization - constrained
#ret = opt.differential_evolution(func=cost, bounds=bnd)

alpha,beta,bt0,tau,rho,delta,btv,ldv,i0 = ret.x; s0 = N-i0-q0-v0-r0-d0; y0 = [s0,i0,1,q0,0,r0,0,v0,d0,n0,g0]
sol = igt.solve_ivp(siqrvd, [0,t_f], y0, method='Radau', args=(alpha,beta,bt0,tau,rho,delta,btv,ldv), t_eval=t_eval)

In [ ]:
# Optimized parameters
#beta=0.0010494808533761054; bt0=0.2517622183943904; btv=1.0120458672251784; ldv=368.56257380772416; i0=11.27466029752497
#alpha=0.004773714726469927; tau=0.0011327215505986237; rho=0.009089198453357721; delta=0.00017997415261673138

Check graphical fit.

In [ ]:
fig = go.Figure(layout=go.Layout(plot_bgcolor='white',xaxis=dict(gridcolor='white'),yaxis=dict(gridcolor='white')))
fig.add_trace(go.Scatter(x=x,y=confirmed,name='Net infections, reported',mode='markers',marker_color='steelblue'))
fig.add_trace(go.Scatter(x=x,y=sol.y[9],name='Net infections, estimated',mode='lines',marker_color='blue'))
fig.add_trace(go.Scatter(x=x,y=vaccinated/10,name='Full vaccinations, reported x10',mode='markers',marker_color='plum'))
fig.add_trace(go.Scatter(x=x,y=sol.y[7]/10,name='Full vaccinations, estimated x10',mode='lines',marker_color='purple'))
fig.add_trace(go.Scatter(x=x,y=recovered[:lrv],name='Recoveries, reported',mode='markers',marker_color='limegreen'))
fig.add_trace(go.Scatter(x=x,y=sol.y[10],name='Recoveries, estimated',mode='lines',marker_color='green'))
fig.add_trace(go.Scatter(x=x,y=deceased/1,name='Deceased, reported',mode='markers',marker_color='pink'))
fig.add_trace(go.Scatter(x=x,y=sol.y[8]/1,name='Deceased, estimated',mode='lines',marker_color='red'))
fig.update_layout(legend=dict(yanchor='bottom',y=0.5,xanchor='left',x=0.2))
fig.update_layout(title='Optimized predictions vs reported data of COVID-19 cases',
                  xaxis_title='Days',yaxis_title='Number of individuals')
#pio.write_image(fig, 'filename.pdf', width=700, height=775)

Compute variant reproduction numbers $R_t=(1-\alpha)\frac{\beta_tS}{(\rho+\tau)N}, R_v=(1-\alpha)\frac{\beta_t(S+R_1)\beta_v}{(\rho+\tau)N}$

In [ ]:
tms = np.arange(t_f)
btt = ((beta+bt0)+(bt0-beta)*np.tanh((tms-300)/100))/2
btt[:lock] = bt0; bvt = btv*np.ones(t_f); bvt[:int(ldv)] = 0
alt = alpha*np.ones(t_f); alt[:lvf] = 0
st = sol.y[0,:t_f]; r1 = sol.y[5,:t_f]
rt = (1-alpha)*btt/(rho+tau)*st/N
rv = (1-alpha)*btt*bvt/(rho+tau)*(st+r1)/N
print("beta=",beta,"beta_0=",bt0,"beta_v=",btv,"ldv=",ldv)
print("alpha=",alpha,"tau=",tau,"rho=",rho,"delta=",delta)
print("Recovery=",1/rho,"I_0=",i0)

beta= 0.0010057409071323594 beta_0= 0.2524820320475236 beta_v= 0.9981152818005994 ldv= 365.39408539006683
alpha= 0.004794490250868523 tau= 0.0011364915658022845 rho= 0.009087550595595503 delta= 0.00019724060641749974
Recovery= 110.04065281184499 I_0= 10.5632759089014


In [ ]:
fig3 = go.Figure(layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig3.add_trace(go.Scatter(x=x, y=sol.y[0]/5e7, name='Susceptible x50M', line=dict(color='blue',width=3)))
fig3.add_trace(go.Scatter(x=x, y=sol.y[1]/1e7, name='Infections, variant-I x10M', line=dict(color='black',width=2)))
fig3.add_trace(go.Scatter(x=x, y=sol.y[2]/1e7, name='Infections, variant-II x10M', line=dict(color='black',width=4)))
fig3.add_trace(go.Scatter(x=x, y=sol.y[3]/1e6, name='Quarantined, variant-I x1M', line=dict(color='red',width=2)))
fig3.add_trace(go.Scatter(x=x, y=sol.y[4]/1e6, name='Quarantined, variant-II x1M', line=dict(color='red',width=4)))
fig3.add_trace(go.Scatter(x=x, y=sol.y[5]/1e7, name='Recoveries, variant-I x10M', line=dict(color='green',width=2)))
fig3.add_trace(go.Scatter(x=x, y=sol.y[6]/1e7, name='Recoveries, variant-II x10M', line=dict(color='green',width=4)))
fig3.add_trace(go.Scatter(x=x, y=sol.y[7]/1e7, name='Fully vaccinated x10M', line=dict(color='purple',width=3,dash='dash')))
fig3.add_trace(go.Scatter(x=x, y=sol.y[8]/1e5, name='Deceased x100K', line=dict(color='orange',width=3,dash='dash')))
fig3.add_trace(go.Scatter(x=[x[lock],x[lock]],y=[0,1],name='Lockdown',mode='lines',marker_color='grey'))
# fig3.update_layout(autosize=False,width=6e2,height=4e2,margin=dict(l=5,r=5,t=50,b=5))
fig3.update_layout(legend=dict(yanchor='bottom',y=0.5,xanchor='left',x=0.1))
fig3.update_layout(title='Dynamics of COVID-19 variants led waves',xaxis_title='Days',yaxis_title='Number of individuals')

In [ ]:
fig4 = go.Figure(layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig4.add_trace(go.Scatter(x=x[lock:], y=rt[lock:], name='Dominant variant-I', marker_color='brown', line=dict(width=4)))
fig4.add_trace(go.Scatter(x=x[lock:], y=rv[lock:], name='Dominant variant-II', marker_color='purple', line=dict(width=4)))
fig4.update_layout(legend=dict(yanchor='bottom',y=0.5,xanchor='left',x=0.1)); fig4.update_xaxes(nticks=7)
fig4.update_layout(title='Time-varying rate of reproduction of variants',xaxis_title='Days',yaxis_title='Rate of reproduction')

Predictions

In [ ]:
days_ahead = 180; ctrl = 1e4
dtx = x + [x[-1]+dt.timedelta(days=day) for day in range(1, days_ahead)]
tfd = t_f+t_f+days_ahead; t_eval = np.arange(0,tfd,1)
sol = igt.solve_ivp(siqrvd, [0,tfd], y0, method='Radau', args=(alpha,beta,bt0,tau,rho,delta,btv,ldv), t_eval=t_eval)

In [ ]:
fig2 = go.Figure(layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig2.add_trace(go.Scatter(x=x,y=confirmed/1e7,name='Net infections, reported x10M',mode='markers',marker_color='steelblue'))
fig2.add_trace(go.Scatter(x=dtx, y=sol.y[9]/1e7, name='Net infections, estimated x10M', marker_color='blue'))
fig2.add_trace(go.Scatter(x=x,y=vaccinated/1e8,name='Full vaccinations, reported x100M',mode='markers',marker_color='plum'))
fig2.add_trace(go.Scatter(x=dtx, y=sol.y[7]/1e8, name='Full vaccinations, estimated x100M', marker_color='purple'))
fig2.add_trace(go.Scatter(x=x,y=recovered[:lrv]/1e7,name='Recoveries, reported x10M',mode='markers',marker_color='limegreen'))
fig2.add_trace(go.Scatter(x=dtx, y=sol.y[10]/1e7, name='Recoveries, estimated x10M', marker_color='green'))
fig2.add_trace(go.Scatter(x=x,y=deceased/1e7,name='Deceased, reported x10M',mode='markers',marker_color='pink'))
fig2.add_trace(go.Scatter(x=dtx, y=sol.y[8]/1e7, name='Deceased, estimated x10M', marker_color='red'))
fig2.add_trace(go.Scatter(x=dtx, y=(sol.y[1]+sol.y[2])/1e8, name='Net active infections x100M', line=dict(color='black',width=3)))
#fig2.add_trace(go.Scatter(x=dtx, y=(sol.y[3]+sol.y[4])/1e6, name='Active Quarantined(1M)', marker_color='brown'))
fig2.add_trace(go.Scatter(x=[dtx[lin],dtx[lin]],y=[0,5],name='Learn | predict',mode='lines',line={'dash':'dash'},marker_color='grey'))
#fig2.update_layout(autosize=False,width=5.5e2,height=4e2,margin=dict(l=5,r=5,t=50,b=5))
fig2.update_layout(legend=dict(yanchor='bottom',y=0.4,xanchor='left',x=0.1))
fig2.update_layout(title='COVID-19 projections based on learnt dynamics',xaxis_title='Days',yaxis_title='Number of individuals')

In [ ]:
fig1 = go.Figure(layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[0]/5e7, name='Susceptible x50M', line=dict(color='blue',width=3)))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[1]/1e7, name='Infections, variant-I x10M', line=dict(color='black',width=2)))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[2]/1e7, name='Infections, variant-II x10M', line=dict(color='black',width=4)))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[3]/1e6, name='Quarantined, variant-I x1M', line=dict(color='red',width=2)))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[4]/1e6, name='Quarantined, variant-II x1M', line=dict(color='red',width=4)))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[5]/1e7, name='Recoveries, variant-I x10M', line=dict(color='green',width=2)))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[6]/1e7, name='Recoveries, variant-II x10M', line=dict(color='green',width=4)))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[7]/5e7, name='Fully vaccinated x50M', line=dict(color='purple',width=3,dash='dash')))
fig1.add_trace(go.Scatter(x=dtx, y=sol.y[8]/1e5, name='Deceased x100K', line=dict(color='orange',width=3,dash='dash')))
fig1.add_trace(go.Scatter(x=[dtx[lock],dtx[lock]],y=[0,1],name='Lockdown',mode='lines',marker_color='grey'))
# fig1.update_layout(autosize=False,width=6e2,height=4e2,margin=dict(l=5,r=5,t=50,b=5))
fig1.update_layout(legend=dict(yanchor='bottom',y=0.5,xanchor='left',x=0.08))
fig1.update_layout(title='Projected evolution of COVID-19 variants',xaxis_title='Days',yaxis_title='Number of individuals')

Lyapunov control: Time-varying, $\alpha_t\geq\max(1-\frac{(\rho+\tau)N}{\beta_tS}, 1-\frac{(\rho+\tau)N}{\beta_t(S+R_1)\beta_v})$ Maximal, $\alpha=1-\frac{\rho+\tau}{\bar\beta\beta_v}$

In [ ]:
ctrl = 510 # begin control
sol = igt.solve_ivp(siqrvd, [0,tfd], y0, method='Radau', args=(alpha,beta,bt0,tau,rho,delta,btv,ldv), t_eval=t_eval)
fig5 = go.Figure(layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[0]/5e7, name='Susceptible x50M', line=dict(color='blue',width=3)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[1]/1e7, name='Infections, variant-I x10M', line=dict(color='black',width=2)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[2]/1e7, name='Infections, variant-II x10M', line=dict(color='black',width=4)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[3]/1e6, name='Quarantined, variant-I x1M', line=dict(color='red',width=2)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[4]/1e6, name='Quarantined, variant-II x1M', line=dict(color='red',width=4)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[5]/1e7, name='Recoveries, variant-I x10M', line=dict(color='green',width=2)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[6]/1e7, name='Recoveries, variant-II x10M', line=dict(color='green',width=4)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[7]/5e7, name='Fully vaccinated x50M', line=dict(color='purple',width=3,dash='dash')))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[8]/1e5, name='Deceased x100K', line=dict(color='orange',width=3,dash='dash')))
fig5.add_trace(go.Scatter(x=[dtx[ctrl],dtx[ctrl]],y=[0,2],name='Feedback control',mode='lines',marker_color='grey'))
# fig5.update_layout(autosize=False,width=6e2,height=4e2,margin=dict(l=5,r=5,t=50,b=5))
fig5.update_layout(legend=dict(yanchor='bottom',y=0.5,xanchor='left',x=0.08))
fig5.update_layout(title='Infection regulation by time-varying feedback control based vaccination',xaxis_title='Days',
                   yaxis_title='Number of individuals')

In [ ]:
amax = 1 - (rho+tau)/(bt0*btv); ctrl = 540
sol = igt.solve_ivp(siqrvd, [0,tfd], y0, method='Radau', args=(alpha,beta,bt0,tau,rho,delta,btv,ldv), t_eval=t_eval)
fig5 = go.Figure(layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[0]/5e7, name='Susceptible x50M', line=dict(color='blue',width=4)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[1]/1e7, name='Infections, variant-I x10M', line=dict(color='black',width=2)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[2]/1e7, name='Infections, variant-II x10M', line=dict(color='black',width=4)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[3]/1e6, name='Quarantined, variant-I x1M', line=dict(color='red',width=2)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[4]/1e6, name='Quarantined, variant-II x1M', line=dict(color='red',width=4)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[5]/1e7, name='Recoveries, variant-I x10M', line=dict(color='green',width=2)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[6]/1e7, name='Recoveries, variant-II x10M', line=dict(color='green',width=4)))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[7]/5e7, name='Fully vaccinated x50M', line=dict(color='purple',width=3,dash='dash')))
fig5.add_trace(go.Scatter(x=dtx, y=sol.y[8]/1e5, name='Deceased x100K', line=dict(color='orange',width=3,dash='dash')))
fig5.add_trace(go.Scatter(x=[dtx[ctrl],dtx[ctrl]],y=[0,1],name='Feedback control',mode='lines',marker_color='grey'))
# fig5.update_layout(autosize=False,width=6e2,height=4e2,margin=dict(l=5,r=5,t=50,b=5))
fig5.update_layout(legend=dict(yanchor='bottom',y=0.5,xanchor='left',x=0.08))
fig5.update_layout(title='Infection regulation by time-invariant feedback control based vaccination',xaxis_title='Days',
                   yaxis_title='Number of individuals')